In [1]:
from blocks.encoder import Encoder, VariationalEncoder
from blocks.pqmf import PQMF
from blocks.decoder import Generator
from blocks.latent_discriminator import LatentDiscriminator
import torch
import pytorch_lightning as pl
from aux import AudioDistanceV1
from dataset_lmdb import AudioDataset
from model import JeffVAE
from config import Config as config
import torch.multiprocessing as mp
import os
from torch.utils.data import DataLoader, Subset, random_split
from lightning.pytorch.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint

/home/jed/miniconda3/envs/SemanticFader/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
full_dataset = AudioDataset(
        db_path='footsteps_sf_db_speed_grass',
        descriptors=config.DESCRIPTORS,
        attribute_name='attributes',
        nb_bins=config.NUM_BINS
    )

computing min and max...
min: -0.12417352199554443, max: 1.1036418676376343 computed
computing bins...
bins computed


In [3]:
def custom_collate(batch):
    audios, features, bin_values = zip(*batch)
    
    # Stack audios and features normally
    audios = torch.stack(audios)
    features = torch.stack(features) 
    
    # bin_values should be the same for all samples, so just take the first one
    # This prevents it from being batched
    bin_values = bin_values[0]  # Take first sample's bin_values
    
    return audios, features, bin_values

In [4]:
device = torch.device('cuda')

# Split dataset (e.g., 80% train, 10% val, 10% test)
train_size = int(0.8 * len(full_dataset))
val_size = int(0.1 * len(full_dataset))
test_size = len(full_dataset) - train_size - val_size
    
train_dataset, val_dataset, test_dataset = random_split(
    full_dataset, 
    [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42)  # For reproducibility
)
    
train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True, num_workers=0, collate_fn=custom_collate)
val_loader = DataLoader(val_dataset, batch_size=config.BATCH_SIZE, shuffle=False, num_workers=0, collate_fn=custom_collate)
test_loader = DataLoader(test_dataset, batch_size=config.BATCH_SIZE, shuffle=False, num_workers=0, collate_fn=custom_collate)
    
#initialize components
encoder = config.ENCODER
variational_encoder = config.VE
decoder = config.DECODER
pqmf = config.PQMF
latent_discriminator = config.LATENT_DISCRIMINATOR
    
#initialize model
model = JeffVAE(
        latent_size=config.LATENT_SIZE,
        encoder=variational_encoder,
        decoder=decoder,
        latent_discriminator=latent_discriminator,
        pqmf=pqmf,
        multiband_audio_distance=AudioDistanceV1,
        learning_rate=1e-3  # Specify learning rate
    )
model.train_loader = train_loader
model = model.to(device)
    
#initialize optimizer
model.configure_optimizers()  # Set up the optimizer for the training step
    
logger = TensorBoardLogger("tb_logs", name="sfRAVE")
print('num epochs: ', config.NUM_EPOCHS)

# Save a checkpoint every 500 training steps
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints",         # Directory to save checkpoints
    filename="sfRAVE-{step}",      # Filename template
    save_top_k=-1,                 
    every_n_train_steps=12000,
    monitor = 'recon_loss'
)
     # Initialize trainer
trainer = pl.Trainer(
        callbacks=[checkpoint_callback],
        max_epochs=config.NUM_EPOCHS,
        accelerator='gpu',  # Will automatically detect available hardware
        devices=1,
        logger=logger, 
        log_every_n_steps=100# Add TensorBoard logging
    )
    
    # Train the model
trainer.fit(model, train_loader, val_loader)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 5090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/home/jed/miniconda3/envs/SemanticFader/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:658: Checkpoint directory /home/jed/SemanticFader/checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                     | Type                | Params | Mode 
-------------------------------------------------------------------------
0 | encoder                  | VariationalEncoder  | 4.8 M  | train
1 | decoder                  | Generator           | 12.8 M | t

num epochs:  50000
Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]allarr shape: torch.Size([16, 1, 2, 130])
bins shape: torch.Size([2, 16])


/home/jed/miniconda3/envs/SemanticFader/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=27` in the `DataLoader` to improve performance.


RuntimeError: The expanded size of the tensor (2) must match the existing size (32) at non-singleton dimension 1.  Target sizes: [16, 2, 130].  Tensor sizes: [32, 130]